In [ ]:
import sqlite3
import os
import pysolr
import re

sqlitedb = os.path.join(os.path.expanduser('~'),'Box Sync', 'GradSchoolStuff', 'MastersProject', 'mimic3', 'mimic3.sqlite')
if not (os.path.exists(sqlitedb)):
    print("Specified database does not exist")
    sys.exit()

connection = sqlite3.connect(sqlitedb)

# Connect to Solr instance.
solr = pysolr.Solr('http://192.168.99.100:8983/solr/radiology')

with connection:
    cur = connection.cursor()
    recordset = cur.execute("""
        SELECT * FROM noteevents
        WHERE description LIKE '%CT%'
            AND category = 'Radiology'
        ORDER BY row_id
        limit 500""")
    col_names = [cn[0] for cn in cur.description]

    # patterns for reason section
    reason_1 = re.compile('^\s*REASON FOR THIS EXAMINATION:\s*$')
    reason_2 = re.compile('^\s*Reason:')
    
    for row in recordset:
        document = {}
        #print('++++++', col_names[idx], ': ', col)
        # want row_id, subject_id, hadm_id, description, text
        document['id'] = row[col_names.index("ROW_ID")]
        document['SUBJECT_ID_i'] = row[col_names.index("SUBJECT_ID")]
        # not all reports have a hospital admission id, so this may not be in Solr
        document['HADM_ID_i'] = row[col_names.index("HADM_ID")]
        # store description as a single string, but also allow for searching parts
        document['DESCRIPTION_s'] = row[col_names.index("DESCRIPTION")]
        document['DESCRIPTION_t'] = row[col_names.index("DESCRIPTION")]
        document['TEXT_t'] = row[col_names.index("TEXT")]
        
        """
        Out of 108,896 reports, only 4 don't use 'REASON FOR THIS EXAMINATION:' 
        or 'Reason:' for the reason for exam section. By manual inspection, those 4
        reports are not ones related to PE nor PAH.
        
        Reports can have multiple reason sections. From inspection of the data,
          * ^'Reason:' (if it exists) is just a one line section and is a portion of the 
            'REASON FOR THIS EXAMINATION:' section (if it exists)
          * ^'REASON FOR THIS EXAMINATION:' (if it exists) is a multi-line section and the actual
            reason starts on the next line. There are 3 cases where this section was repeated and
            in some of the repeated locations the reason may partially be on the same line.
        
        """
        reason_short = ''
        reason_long = ''
        if 'REASON FOR THIS EXAMINATION:' in row[col_names.index("TEXT")] or 'Reason:' in row[col_names.index("TEXT")]:
            found_multiline = ''
            for line in row[col_names.index("TEXT")].rstrip().split('\n'):
                # multi-line reason section start -------------
                if reason_1.match(line) is not None:
                    found_multiline = True
                # single line reason section -------------
                elif reason_2.match(line) is not None:
                    reason_short = line.replace('Reason:', '').strip()
                # multi-line reason section continuation -------------
                elif found_multiline:
                    # should add to reason_long until end of section
                    if re.match('^\s\s\w*', line) is not None:
                        reason_long += line
                    else:
                        found_multiline = False
            if len(reason_long.strip()) > 0:
                document['REASON_t'] = reason_long.strip()
                #print(row[col_names.index("ROW_ID")], 'reason_long', reason_long.strip())
            else:
                document['REASON_t'] = reason_short
                #print(row[col_names.index("ROW_ID")], 'reason_short', reason_short)
        else:
            print('------------reason missing for', row[col_names.index("ROW_ID")])
        
        '''
        Identify impression section. Generally starts with 'IMPRESSION:' followed by newline, then text
        found some that have colon on next line...
        
        Found some have 'CONCLUSION' instead of 'IMPRESSION'
        
        Some missing IMPRESSION and CONCLUSION, just have FINDINGS section. Should that be a substitute?
        
        wierd example - instead of IMPRESSION, has this:
         ______________________________________________________________________________
                                  PFI REPORT
        '''
        if 'IMPRESSION' in row[col_names.index("TEXT")] or 'CONCLUSION' in row[col_names.index("TEXT")]:
            print(row[col_names.index("ROW_ID")], 'found impression section')
        else:
            print(row[col_names.index("ROW_ID")], '------------impression missing')
        
        
        #print('Adding row_id', row[col_names.index("ROW_ID")], 'to Solr.')
        #solr.add([document])


In [ ]:
#Now need to devide reports into sections...
# Or should this be done before importing into Solr???

# Right now I think that section segmentation should be done before solr insertion as the section
# can be inserted as another data field.
# So far, Reason and Impression sections are those that I think most relevant for looking for PE/PAH
# of course need to see how many reports actually have an impressions section.